In [16]:
import sys
sys.path.append('/Users/wintercyan/code/TreeSeg/notebooks/')
import tensorflow as tf
import numpy as np
from PIL import Image
import rasterio
import imgaug as ia
from imgaug import augmenters as iaa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import rasterio.warp             # Reproect raster samples
from functools import reduce
from tensorflow.keras.models import load_model

from core.UNet import UNet
from core.losses import tversky, accuracy, dice_coef, dice_loss, specificity, sensitivity
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.frame_info import FrameInfo
from core.dataset_generator import DataGenerator
from core.split_frames import split_dataset
from core.visualize import display_images

from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools
import matplotlib.patches as patches
from matplotlib.patches import Polygon

import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print(tf.__version__)
tf.config.list_physical_devices('GPU')
# from configx import Preprocessing_prediction
# config = Preprocessing_prediction.Configuration(folder="full_area3")


2.6.2


[]

In [19]:
# Define the model and compile it
OPTIMIZER = adaDelta
LOSS = tversky 

#Only for the name of the model in the very end
OPTIMIZER_NAME = 'AdaDelta'
LOSS_NAME = 'weightmap_tversky'
model_path = "/Users/wintercyan/code/TreeSeg/notebooks/saved_models/UNet/model-init.h5"
model = load_model(model_path, custom_objects={'tversky': LOSS, 'dice_coef': dice_coef, 'dice_loss':dice_loss, 'accuracy':accuracy , 'specificity': specificity, 'sensitivity':sensitivity}, compile=False)

In [20]:
def read_method(method:str=''):
    if method == 'Image' or method == '':
        return Image.open
    if method == 'rasterio':
        return rasterio.open
    if method == 'numpy':
        return np.load

In [6]:

write_dir="/home/lenovo/treeseg-dataset/predictions/model1115"
dataset_dir = "/home/lenovo/treeseg-dataset/full_process/sample_128"

areas = [1, 50, 220]
rows = [0,1,2,3]
cols = [0,1,2,3]
for a in areas:
    for r in rows:
        for c in cols:
            ndvi_img= read_method()(dataset_dir+f"pan-0-area{a}-r{r}c{c}.png")
            pan_img= read_method()(dataset_dir+f"ndvi-0-area{a}-r{r}c{c}.png")

            comb_img = np.stack((ndvi_img, pan_img), axis=0)
            comb_img = np.transpose(comb_img, axes=(1,2,0)) #Channel at the end
            comb_img = comb_img[np.newaxis,:]
            prediction = model.predict(comb_img, steps=1)
            prediction[prediction>0.5]=1
            prediction[prediction<=0.5]=0
            
            p=np.concatenate((comb_img , prediction), axis = -1)
            print(prediction.shape)
            display_images(p,titles=None, interpolation=None, save=True, dir=f"{write_dir}/area{a}-c{r}r{c}")

NameError: name 'Image' is not defined